# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [13]:
import pandas as pd
import math
import numpy as np
import scipy.stats as scs

In [3]:
job_disc = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
pd.set_option('display.max_columns', 65)
job_disc.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [84]:
job_disc.shape

(4870, 65)

## 1. Which test is appropriate for this problem? Does CLT apply? 

The Chi-square test for independence is appropriate for this problem. The CLT applies, based on the expected 
values for success and failure below. 

In [93]:
#generate 2x2 table, check for missing values in variables of interest -- no missingness here
job_disc['call'] = job_disc['call'].astype(int)
pd.crosstab(index = job_disc.race, columns = job_disc.call, margins = True)

call,0,1,All
race,,,
b,2278,157,2435
w,2200,235,2435
All,4478,392,4870


In [1]:
#check that expected values >= 5 (only need to check smallest)
(392*2435) / 4870

196.0

## 2. What are the null and alternate hypotheses?

The null hypothesis is that there is no relationship between whether a person's name sounds black or white and whether or not that person receives a call from employers (i.e., the two events are independent). The alternative is that receiving a call from employers depends on whether a person's name sounds black or white.

## 3. Compute margin of error, confidence interval, and p-value.

- The 95% confidence interval for the difference in proportions for black-sounding names receiving and not receiving a call is (.0571, .1582), with a standard error of 0.0258.
- The p-value is 5e-5, or 0.00005. 

In [91]:
#confidence interval for difference between probabilities for getting a call from employer

o1 = len(job_disc[(job_disc['call'] < 1) & (job_disc['race'] == 'b')]) #number of non-calls for black-sounding names
n1 = len(job_disc[(job_disc['call'] < 1)]) #total resumes not receiving calls
p1 = (o1 + 1)/(n1 + 2)

o2 = sum(job_disc[job_disc.race == 'b'].call) #number of calls for black-sounding names
n2 = sum(job_disc.call) #total number of calls
p2 = (o2 + 1)/(n2 + 2)

SE = math.sqrt(((p1*(1-p1))/(n1+2)) + ((p2*(1-p2))/(n2+2)))

((p1 - p2) - 1.96*SE), ((p1 - p2) + 1.96*SE), SE

(0.057129861440035061, 0.15825039599288737, 0.02579605473287049)

In [25]:
#perform chi-square test and find p-value

    #write function to create contingency table 
def chisq_cols(df, c1, c2):
    groupsizes = df.groupby([c1, c2]).size()
    ctsum = groupsizes.unstack(c1) 
    return(scs.chi2_contingency(ctsum))

    #run test
chisq_cols(job_disc, 'race', 'call')

(16.449028584189371, 4.9975783899632552e-05, 1, array([[ 2239.,  2239.],
        [  196.,   196.]]))

## 4. Write a story describing the statistical significance in the context of the original problem.

Since the p-value (5e-5) is less than .05, we reject the null hypothesis and conclude that there is a significant difference between the probabilities of a white-sounding name and a black-sounding name getting a call from an employer.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

My analysis does not mean that the race associated with one's name is the most important factor because a hypothesis test only indicates that the observed result is not due to chance (i.e., it is statistically significant). That result, however, does not speak to whether or not one variable is important in determining another; it simply indicates the further investigation might be worthwhile. I would need to conduct a controlled, designed experiment to determine which variable is most important.